In [180]:
inFile = open("input", 'r') 
lines = inFile.readlines()
lines = [l.replace('\n', '').replace(' ', '') for l in lines]

sampleLines = [ 
    "2*3+(4*5)", 
    "5+(8*3+9+3*4*3)",
    "5*9*(7*3*3+9*3+(8+6*4))",
    "((2+4*9)*(6+9*8+6)+6)+2+4*2"
]

def eval(operand1, operand2, operation): 
    if operation == '+': 
        return operand1 + operand2
    elif operation == '*': 
        return operand1 * operand2
    else: 
        print(operand1, operation, operand2, " - invalid operation evaluation")

In [196]:
def tokenize(line): 
    '''
    Tokenizes the line into operands, operations and expressions
    '''
    lineList = list(line)
    tokens = []
    newNum = []
    newParan = []
    paranNode = {
        "leftCurl":0,
        "rightCurl":0, 
        "string":[]
    }
    paranFlag = False
    digitFlag = False
    while lineList:
        c = lineList.pop(0)
        if digitFlag and not paranFlag: 
            if c.isdigit():
                newNum.append(c)
            else: 
                digitFlag = False 
                tokens.append(''.join(newNum))
                newNum = []
        if paranFlag: 
            if c == '(': 
                paranNode['leftCurl'] += 1
                paranNode['string'].append(c)
            elif c == ')': 
                # check if end
                paranNode['rightCurl'] += 1
                if paranNode['leftCurl'] == paranNode['rightCurl']: 
                    # print("end paran")
                    paranFlag = False
                    tokens.append(tokenize(paranNode['string']))                    
                    paranNode['string'] = []
                    paranNode['leftCurl'] = 0
                    paranNode['rightCurl'] = 0
                else: 
                    paranNode['string'].append(c)
            else: 
                paranNode['string'].append(c)
        elif c == '(':
            paranFlag = True
            paranNode['leftCurl'] += 1
        elif c.isdigit():
            if not digitFlag:
                digitFlag = True
                newNum.append(c)
            if not lineList:
                tokens.append(''.join(newNum))
        else: 
            tokens.append(c)
    return tokens

In [298]:
def evalExpression(tokens): 
    listIndices = [tokens.index(c) for c in tokens if isinstance(c, list)]
    for i in listIndices:         
        expression = tokens[i]
        evaluated = evalExpression(expression)
        tokens[i] = evaluated


    for op in ['+', '*']: 
        while tokens.count(op) != 0: 
            i = tokens.index(op)
            operand1 = tokens[i-1]
            operand2 = tokens[i+1]
            if operand1.isdigit() and operand2.isdigit(): 
                evaluated = eval(int(operand1), int(operand2), op)
                tokens[i] = str(evaluated)
                del tokens[i-1]
                del tokens[i]
                opIndices = [tokens.index(c) for c in tokens if c == op]
    
    return tokens[0]
                
        

In [299]:
tokens = tokenize("5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))".replace(' ', ''))

# tokens = tokenize("(1+2)*(3)")
print(tokens)
evalExpression(tokens)

['5', '*', '9', '*', ['7', '*', '3', '*', '3', '+', '9', '*', '3', '+', ['8', '+', '6', '*', '4']]]


'669060'

In [301]:
for s in sampleLines:
    tokens = tokenize(s)
    print(evalExpression(tokens))

46
1445
669060
23340


In [303]:
total = 0
for s in lines: 
    tokens = tokenize(s)
    total += int(evalExpression(tokens))
print(total)

218621700997826
